Some of the following implementation derive from:
[1] Official document of pytorch
[2] LRN implementation [here](https://zhuanlan.zhihu.com/p/29786939) 

In [1]:
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as T
import torch.utils.data as data
import torch

In [11]:
# For now, there is no official implementation in pytorch.
# see the pr here: https://github.com/pytorch/pytorch/issues/653
class LRN(nn.Module):
    def __init__(self, local_size=1, alpha=1.0, beta=0.75, ACROSS_CHANNELS=False):
        super(LRN, self).__init__()
        self.ACROSS_CHANNELS = ACROSS_CHANNELS
        if self.ACROSS_CHANNELS:
            self.average=nn.AvgPool3d(kernel_size=(local_size, 1, 1), #0.2.0_4会报错，需要在最新的分支上AvgPool3d才有padding参数
                    stride=1,
                    padding=(int((local_size-1.0)/2), 0, 0)) 
        else:
            self.average=nn.AvgPool2d(kernel_size=local_size,
                    stride=1,
                    padding=int((local_size-1.0)/2))
        self.alpha = alpha
        self.beta = beta
    
    
    def forward(self, x):
        if self.ACROSS_CHANNELS:
            div = x.pow(2).unsqueeze(1)
            div = self.average(div).squeeze(1)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)#这里的1.0即为bias
        else:
            div = x.pow(2)
            div = self.average(div)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        x = x.div(div)
        return x

In [14]:
class AlexNet(nn.Module):
    def __init__(self, num_classes = 1000): #imagenet数量
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            LRN(local_size=5, alpha=1e-4, beta=0.75, ACROSS_CHANNELS=True)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, groups=2, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            LRN(local_size=5, alpha=1e-4, beta=0.75, ACROSS_CHANNELS=True)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=384, padding=1, kernel_size=3),
            nn.ReLU(inplace=True)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
         #需要针对上一层改变view
        self.layer6 = nn.Sequential(
            nn.Linear(in_features=6*6*256, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout()
        )
        self.layer7 = nn.Sequential(
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout()
        )
        
        self.layer8 = nn.Linear(in_features=4096, out_features=num_classes)
        
    def forward(self, x):
        x = self.layer5(self.layer4(self.layer3(self.layer2(self.layer1(x)))))
        x = x.view(-1, 6*6*256)
        x = self.layer8(self.layer7(self.layer6(x)))
        
        return x

In [2]:
class AlexNet_standard(nn.Module):

    def __init__(self, num_classes=10):
        super(AlexNet_standard, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [9]:
# set all the hyper-params here
batch_size = 64
model_type = 'standard'  #'customized'

In [5]:
# define dataset
transform = T.Compose([
                T.Resize(40),
                T.RandomCrop(32),
                T.RandomHorizontalFlip(),
                T.ToTensor(),
            ])
# /Users/hyli/Project/ssd.pytorch/data
# ~/project/ssd.pytorch/data
dataset = dset.CIFAR10(root='/Users/hyli/Project/ssd.pytorch/data', train=True, transform=transform, download=True)
dataset.num_classes = 10
dataset.name = 'cifar10'
train_loader = data.DataLoader(dataset, batch_size, num_workers=2, shuffle=True)

Files already downloaded and verified


In [15]:
def weight_init(m):
    # 使用isinstance来判断m属于什么类型
    if isinstance(m, nn.Conv2d):
        import math
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
    elif isinstance(m, nn.BatchNorm2d):
        # m中的weight，bias其实都是Variable，为了能学习参数以及后向传播
        m.weight.data.fill_(1)
        m.bias.data.zero_()

# define model
model = AlexNet(dataset.num_classes) if model_type == 'customized' else AlexNet_standard(dataset.num_classes)
model_info = 'use customized model with LRN' if model_type == 'customized' else 'use standard alexnet model'
print(model_info)
model.apply(weight_init)

use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()
    print('USE GPU')
else:
    print('USE CPU')

# define loss
criterion = nn.CrossEntropyLoss(size_average=False)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.99))

TypeError: __init__() got an unexpected keyword argument 'padding'

In [8]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_gpu:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            

for epoch in range(1, 5):
    train(epoch)
print('Training finished!')

Train Epoch: 1 [0/50000 (0%)]	Loss: 137.541626
Train Epoch: 1 [640/50000 (1%)]	Loss: 139.267303
Train Epoch: 1 [1280/50000 (3%)]	Loss: 130.379684
Train Epoch: 1 [1920/50000 (4%)]	Loss: 128.870026
Train Epoch: 1 [2560/50000 (5%)]	Loss: 128.134903
Train Epoch: 1 [3200/50000 (6%)]	Loss: 131.081665


Process Process-4:
Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/hyli/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/hyli/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/hyli/anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
    self._target(*self._args, **self._kwargs)
  File "/Users/hyli/anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/Users/hyli/anaconda/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    self._target(*self._args, **self._kwargs)
    r = index_queue.get()
  File "/Users/hyli/anaconda/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
  File "/Users/hyli/anaconda/lib/python2.7/multiprocessing/queues.py", line 378, in get
    r = index_queue.get()
    return recv()
  File "/Use

KeyboardInterrupt: 

What we do above is just the very basic of the alexnet on CIFAR-10. There are defenitely many aspects that you can explore and we will cover some of the following topics (solutions, results) in the next week:
[1] different learning strategies (adam, rmsprop, etc.)
[2] bigger batch size really helpful?
[3] remove FC and replace it with fully convolutional layers
[4] data augmentation (e.g., multi-crop at test stage)

TODO: embed visdom to visualize loss during training.